In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pywt
from scipy import signal
from sklearn.preprocessing import RobustScaler, LabelEncoder
import torch
 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.signal import butter, filtfilt
from scipy.linalg import eigh
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif



import warnings
warnings.filterwarnings('ignore')

# ============================
# Step 1: Data Loading with Preprocessing (SSVEP only)
# ============================
def load_index_csvs(base_path):
    train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
    validation_df = pd.read_csv(os.path.join(base_path, 'validation.csv'))
    test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

    le_ssvep = LabelEncoder()

    ssvep_train_labels = train_df[train_df['task'] == 'SSVEP']['label']
    if len(ssvep_train_labels) > 0:
        le_ssvep.fit(ssvep_train_labels)

        for df in [train_df, validation_df]:
            if 'label' in df.columns:
                ssvep_mask = df['task'] == 'SSVEP'
                if ssvep_mask.any():
                    df.loc[ssvep_mask, 'label'] = le_ssvep.transform(df.loc[ssvep_mask, 'label'])

    return train_df, validation_df, test_df, le_ssvep


# ============================
# FBCCA Feature Extractor
# ============================

class FBCCAExtractor:
    def __init__(self, fs=250, num_harmonics=2, num_subbands=5):
        self.fs = fs
        self.num_harmonics = num_harmonics
        self.num_subbands = num_subbands
        self.target_freqs = [7, 8, 10, 13]  # SSVEP targets
        self.subbands = [
            (5, 40), (6, 38), (7, 36), (8, 34), (9, 32)
        ][:num_subbands]

    def _bandpass_filter(self, data, low_freq, high_freq, order=4):
        nyq = 0.5 * self.fs
        low = low_freq / nyq
        high = high_freq / nyq
        b, a = butter(order, [low, high], btype='band')
        return filtfilt(b, a, data, axis=0)

    def _generate_reference_signals(self, freq, n_samples):
        t = np.arange(n_samples) / self.fs
        ref = [
            np.sin(2 * np.pi * freq * i * t) for i in range(1, self.num_harmonics+1)
        ] + [
            np.cos(2 * np.pi * freq * i * t) for i in range(1, self.num_harmonics+1)
        ]
        return np.stack(ref, axis=1)

    def _cca_correlation(self, X, Y):
        cca = CCA(n_components=1)
        cca.fit(X, Y)
        X_c, Y_c = cca.transform(X, Y)
        return np.corrcoef(X_c.T, Y_c.T)[0, 1]

    def extract_fbcca_features(self, eeg_data):
        n_samples = eeg_data.shape[0]
        corrs = []

        for low, high in self.subbands:
            filtered = self._bandpass_filter(eeg_data, low, high)
            sub_corrs = [
                self._cca_correlation(filtered, self._generate_reference_signals(freq, n_samples))
                for freq in self.target_freqs
            ]
            corrs.append(sub_corrs)

        corrs = np.array(corrs)
        weights = 1 / np.arange(1, self.num_subbands + 1)
        weights /= weights.sum()
        return np.dot(weights, corrs)  # shape: (num_targets,)

# ============================
# Feature Extractor Using FBCCA Only
# ============================
class EEGFeatureExtractor:
    def __init__(self, fs=250):
        self.fs = fs
        self.eeg_channels = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
        self.fbcca_extractor = FBCCAExtractor(fs=fs)

    def extract_features(self, trial_data):
        # تأكد إن كل القنوات موجودة
        available_channels = [ch for ch in self.eeg_channels if ch in trial_data.columns]
        if not available_channels:
            raise ValueError("No valid EEG channels found in trial data")

        eeg_data = trial_data[available_channels].values

        try:
            fbcca_feats = self.fbcca_extractor.extract_fbcca_features(eeg_data)
        except Exception as e:
            print(f"FBCCA failed: {e}")
            fbcca_feats = np.zeros(len(self.fbcca_extractor.target_freqs))

        return fbcca_feats


# ============================
# Data Loading with Features
# ============================
def load_trial_data_with_features(row, base_path, feature_extractor):
    dataset = 'train' if row['id'] <= 4800 else 'validation' if row['id'] <= 4900 else 'test'
    eeg_path = os.path.join(base_path, row['task'], dataset, str(row['subject_id']), str(row['trial_session']), 'EEGdata.csv')

    if not os.path.exists(eeg_path):
        raise FileNotFoundError(f"File not found: {eeg_path}")

    eeg_data = pd.read_csv(eeg_path)
    trial = int(row['trial'])
    start = (trial - 1) * 1750
    end = start + 1750
    trial_data = eeg_data.iloc[start:end].copy()

    features = feature_extractor.extract_features(trial_data)
    result = {'id': row['id'], 'features': features, 'task': row['task']}
    if 'label' in row and pd.notna(row['label']):
        result['label'] = row['label']
    return result


def load_all_split_data_with_features(split_df, base_path, task='SSVEP'):
    all_trials = []
    split_df = split_df[split_df['task'] == task]
    extractor = EEGFeatureExtractor()

    for _, row in tqdm(split_df.iterrows(), total=len(split_df), desc=f"Extracting features for {task}"):
        try:
            trial = load_trial_data_with_features(row, base_path, extractor)
            all_trials.append(trial)
        except Exception as e:
            print(f"[Warning] Trial {row['id']} failed: {e}")

    if not all_trials:
        return None, None, None

    features = np.array([t['features'] for t in all_trials])
    labels = np.array([t['label'] for t in all_trials if 'label' in t])
    ids = np.array([t['id'] for t in all_trials])

    print(f"✓ Loaded {len(features)} samples with {features.shape[1]} features")
    return features, labels, ids


# ============================
# Step 4: Deep Learning Models (SSVEP only)
# ============================
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------------------------
# Attention Module for Feature Enhancement
# ----------------------------------
class AttentionModule(nn.Module):
    def __init__(self, input_dim):
        super(AttentionModule, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.ReLU(),
            nn.Linear(input_dim // 2, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        weights = self.attention(x)
        return x * weights

# ----------------------------------
# MLP-Based Classifier with Attention
# ----------------------------------
class EnhancedFeatureClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EnhancedFeatureClassifier, self).__init__()

        hidden_dim1 = max(256, input_dim // 4)
        hidden_dim2 = max(128, input_dim // 8)
        hidden_dim3 = max(64, input_dim // 16)

        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.BatchNorm1d(hidden_dim1),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim1, hidden_dim2),
            nn.BatchNorm1d(hidden_dim2),
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Linear(hidden_dim2, hidden_dim3),
            nn.BatchNorm1d(hidden_dim3),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.attention = AttentionModule(hidden_dim3)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim3, hidden_dim3 // 2),
            nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(hidden_dim3 // 2, num_classes)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.attention(x)
        return self.classifier(x)

# ----------------------------------
# Transformer-Based SSVEP Classifier
# ----------------------------------
class SSVEPFormer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=4, num_layers=2, dropout=0.1):
        super(SSVEPFormer, self).__init__()

        self.embedding = nn.Linear(input_dim, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, 1, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = x + self.pos_embedding
        x = self.transformer(x)
        x = x.squeeze(1)
        return self.classifier(x)

# ----------------------------------
# CNN-Based Classifier
# ----------------------------------
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(16)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (B, 1, input_dim)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return self.fc(x)

# ----------------------------------
# EEGNet Architecture
# ----------------------------------
class EEGNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EEGNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.dropout1(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout2(F.relu(self.bn2(self.fc2(x))))
        return self.fc3(x)

# ----------------------------------
# BiLSTM-Based Classifier
# ----------------------------------
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, num_layers=1):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, 
                            batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (B, seq_len=1, input_dim)
        _, (h_n, _) = self.lstm(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)  # Concatenate forward and backward
        return self.fc(h)

# ----------------------------------
# Transformer + LSTM Hybrid Model
# ----------------------------------
class TransformerLSTMHybrid(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=4, num_layers=1, lstm_hidden=64):
        super(TransformerLSTMHybrid, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.lstm = nn.LSTM(d_model, lstm_hidden, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.transformer(x)
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        return self.classifier(out)

# ============================
# Step 5: Feature-based Dataset
# ============================
class FeatureDataset(Dataset):
    def __init__(self, features, labels=None, ids=None):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels) if labels is not None else None
        self.ids = ids
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]
        else:
            return self.features[idx], self.ids[idx] if self.ids is not None else idx

# ============================
# Step 6: Training for SSVEP
# ============================
def train_model_advanced(model, train_loader, val_loader, epochs=50, lr=1e-3):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training SSVEP model on device: {device}")
    
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr * 0.8, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)
    max_patience = 15

    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct_train = 0
        total_train = 0
        
        for batch in train_loader:
            if len(batch) == 2:
                x, y = batch
                x, y = x.to(device), y.to(device)
            else:
                continue

            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)

            # L2 regularization
            l2_reg = torch.tensor(0.).to(device)
            for param in model.parameters():
                l2_reg += torch.norm(param)
            loss += 0.0001 * l2_reg

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == y).sum().item()
            total_train += y.size(0)

        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        val_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                if len(batch) == 2:
                    x, y = batch
                    x, y = x.to(device), y.to(device)
                else:
                    continue

                outputs = model(x)
                loss = criterion(outputs, y)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == y).sum().item()
                total_val += y.size(0)

        train_acc = correct_train / total_train if total_train > 0 else 0
        val_acc = correct_val / total_val if total_val > 0 else 0

        scheduler.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}")
            print(f"  Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}")
            print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_SSVEP_model.pth')
        else:
            patience_counter += 1

        if patience_counter >= max_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    if os.path.exists('best_SSVEP_model.pth'):
        model.load_state_dict(torch.load('best_SSVEP_model.pth'))
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    return model

def predict_model_advanced(model, test_loader, le=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    predictions = []
    ids = []
    probabilities = []

    with torch.no_grad():
        for batch in test_loader:
            if len(batch) == 2:
                x, id_batch = batch
            else:
                x = batch
                id_batch = list(range(len(x)))

            x = x.to(device)
            outputs = model(x)
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            probabilities.extend(probs.cpu().numpy())

            processed_ids = []
            for id_val in id_batch:
                if torch.is_tensor(id_val):
                    processed_ids.append(int(id_val.item()))
                else:
                    processed_ids.append(int(id_val))
            ids.extend(processed_ids)

    if le is not None:
        predictions = le.inverse_transform(predictions)

    return pd.DataFrame({
        'id': ids,
        'label': predictions,
        'confidence': [max(prob) for prob in probabilities]
    })

# ============================
# Step 7: Main Pipeline for SSVEP Only
# ============================

def get_model(model_name, input_dim, num_classes):
    if model_name == "EnhancedFeatureClassifier":
        return EnhancedFeatureClassifier(input_dim, num_classes)
    elif model_name == "SSVEPFormer":
        return SSVEPFormer(input_dim, num_classes)
    elif model_name == "CNNClassifier":
        return CNNClassifier(input_dim, num_classes)
    elif model_name == "EEGNet":
        return EEGNet(input_dim, num_classes)
    elif model_name == "BiLSTMClassifier":
        return BiLSTMClassifier(input_dim, num_classes)
    elif model_name == "TransformerLSTMHybrid":
        return TransformerLSTMHybrid(input_dim, num_classes)
    else:
        raise ValueError(f"Unknown model name: {model_name}")

def main():
    base_path = '/kaggle/input/mtcaic3'  # Update this if needed

    print("Loading index files...")
    train_df, val_df, test_df, le_ssvep = load_index_csvs(base_path)


    print(f"Train samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples: {len(test_df)}")

    print("\nTask distribution:")
    print("Training:", train_df.groupby('task').size())
    print("Validation:", val_df.groupby('task').size())
    print("Test:", test_df.groupby('task').size())

    all_predictions = []

    task = 'SSVEP'
    print(f"\n{'='*50}")
    print(f"Processing {task} Task")
    print(f"{'='*50}")

    num_classes = 4
    le = le_ssvep

    print(f"Loading {task} training data...")
    train_features, train_labels, train_ids = load_all_split_data_with_features(
        train_df, base_path, task=task
    )

    print(f"Loading {task} validation data...")
    val_features, val_labels, val_ids = load_all_split_data_with_features(
        val_df, base_path, task=task
    )

    print(f"Loading {task} test data...")
    test_features, _, test_ids = load_all_split_data_with_features(
        test_df, base_path, task=task
    )

    if train_features is None or val_features is None or test_features is None:
        print(f"Skipping {task} due to data loading issues")
        return

    print("Scaling features...")
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    val_features_scaled = scaler.transform(val_features)
    test_features_scaled = scaler.transform(test_features)

    print("Selecting best features...")
    selector = SelectKBest(f_classif, k=min(500, train_features_scaled.shape[1]))
    train_features_selected = selector.fit_transform(train_features_scaled, train_labels)
    val_features_selected = selector.transform(val_features_scaled)
    test_features_selected = selector.transform(test_features_scaled)

    print(f"Selected {train_features_selected.shape[1]} features out of {train_features_scaled.shape[1]}")

    train_dataset = FeatureDataset(train_features_selected, train_labels)
    val_dataset = FeatureDataset(val_features_selected, val_labels)
    test_dataset = FeatureDataset(test_features_selected, ids=test_ids)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    input_dim = train_features_selected.shape[1]
    model_name = "EnhancedFeatureClassifier"  # ← غير الاسم لأي نموذج آخر لتجربته
    model = get_model(model_name, input_dim, num_classes)
   

    print(f"Model architecture for {task}:")
    print(f"  Input dimension: {input_dim}")
    print(f"  Number of classes: {num_classes}")
    print(f"  Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    print(f"\nTraining {task} model...")
    model = train_model_advanced(
        model, train_loader, val_loader, epochs=60, lr=0.0008
    )

    print(f"Making predictions for {task}...")
    task_predictions = predict_model_advanced(model, test_loader, le)
    task_predictions['task'] = task
    all_predictions.append(task_predictions)

    print(f"{task} predictions completed. Sample predictions:")
    print(task_predictions.head())
    print(f"Confidence stats: mean={task_predictions['confidence'].mean():.3f}, "
          f"std={task_predictions['confidence'].std():.3f}")

    print(f"\n{'='*50}")
    print("Combining Predictions")
    print(f"{'='*50}")

    final_predictions = pd.concat(all_predictions, ignore_index=True)
    final_predictions = final_predictions.sort_values('id')

    submission = final_predictions[['id', 'label']].copy()
    submission_path = 'submission.csv'
    submission.to_csv(submission_path, index=False)

    print(f"Submission file saved as: {submission_path}")
    print(f"Total predictions: {len(submission)}")
    print("\nFinal prediction distribution:")
    print(final_predictions.groupby(['task', 'label']).size())

    print("\nSubmission preview:")
    print(submission.head(10))
    print("...")
    print(submission.tail(10))

    expected_test_ids = set(test_df['id'].values)
    predicted_ids = set(submission['id'].values)

    if expected_test_ids == predicted_ids:
        print("✓ All test IDs have predictions")
    else:
        missing_ids = expected_test_ids - predicted_ids
        extra_ids = predicted_ids - expected_test_ids
        if missing_ids:
            print(f"⚠ Missing predictions for IDs: {missing_ids}")
        if extra_ids:
            print(f"⚠ Extra predictions for IDs: {extra_ids}")

if __name__ == "__main__":
    main()


Loading index files...
Train samples: 4800
Validation samples: 100
Test samples: 100

Task distribution:
Training: task
MI       2400
SSVEP    2400
dtype: int64
Validation: task
MI       50
SSVEP    50
dtype: int64
Test: task
MI       50
SSVEP    50
dtype: int64

Processing SSVEP Task
Loading SSVEP training data...


Extracting features for SSVEP: 100%|██████████| 2400/2400 [05:41<00:00,  7.04it/s]


✓ Loaded 2400 samples with 4 features
Loading SSVEP validation data...


Extracting features for SSVEP: 100%|██████████| 50/50 [00:06<00:00,  7.65it/s]


✓ Loaded 50 samples with 4 features
Loading SSVEP test data...


Extracting features for SSVEP: 100%|██████████| 50/50 [00:06<00:00,  7.69it/s]


✓ Loaded 50 samples with 4 features
Scaling features...
Selecting best features...
Selected 4 features out of 4
Model architecture for SSVEP:
  Input dimension: 4
  Number of classes: 4
  Model parameters: 701,508

Training SSVEP model...
Training SSVEP model on device: cpu
Epoch 5/60
  Train Loss: 1.0362, Train Acc: 0.5783
  Val Loss: 1.3054, Val Acc: 0.3600
  LR: 0.000320
Epoch 10/60
  Train Loss: 1.0022, Train Acc: 0.5863
  Val Loss: 1.3026, Val Acc: 0.3800
  LR: 0.000640
Epoch 15/60
  Train Loss: 1.0151, Train Acc: 0.5962
  Val Loss: 1.3025, Val Acc: 0.4400
  LR: 0.000320
Epoch 20/60
  Train Loss: 0.9759, Train Acc: 0.6104
  Val Loss: 1.3238, Val Acc: 0.3800
  LR: 0.000640
Epoch 25/60
  Train Loss: 0.9940, Train Acc: 0.6017
  Val Loss: 1.3358, Val Acc: 0.3800
  LR: 0.000320
Early stopping at epoch 29
Best validation accuracy: 0.4600
Making predictions for SSVEP...
SSVEP predictions completed. Sample predictions:
     id     label  confidence   task
0  4951  Backward    0.925140  SS